# Determining the optimal number of hidden layers and neurons for an Artificial Neuron Network(ANN)

This can be challending and often requires experimentation. However , Ther are some of Guidelines and methods taht can help yoi in amking an infomed decison :
- Start Simple : Begin with a simple architecture and gradully increase complexity if needed.
- Gris Search/ Random Search : Use grid search or random search to try sifferent achitectures.
- Cross-Validation : Use cross- Validation to evalute the performance of different achitectures.
- Heuristic and Rule of Thumb : Some heuristic and emperical rules can provide starting point, such as :
    - The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
    - A common practice is to start with 1-2 hidden layers. 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
 

In [ ]:
data = pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])


onehot_encoder = OneHotEncoder(handle_unknown='ignore')
geo_encoder = onehot_encoder.fit_transform(data[['Geography']])

geo_encoder_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder.get_feature_names_out(['Geography']))
data.head()



In [ ]:
onehot_encoder = OneHotEncoder()
geo_encoder = onehot_encoder.fit_transform(data[['Geography']])
geo_encoder


In [ ]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encoder_df

In [ ]:
## Combine One hot encoder column with the original data
data = pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

In [ ]:
X = data.drop('Exited',axis=1)
y = data['Exited']

## Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

##Scale these feature
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
## Define a function to create the model and try differnt parameters(Keras Classifier)

def create_model(neurons = 32 , layers=1):
    model = Sequential()

    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
        
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer = 'adam',loss="binary_crossentropy", metrics=['accuracy'])

    return model

In [ ]:
# Create a Keras Classifier

model = KerasClassifier(neurons = 32 , layers=1,build_fn = create_model, verbose = 1 )

In [ ]:
# Define the grid Search Parameters
param_grid = {
    'neurons' : [16,32,64,128],
    'layers':[1,2,3],
    'epochs':[50,100]
}

In [ ]:
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs =-1,cv=3,verbose=1)
grid_result = grid.fit(X_train,y_train)

#Print The best parameters

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))